In [2]:
text = """
Overview of LangChain v0.2
What’s new in LangChain?

The following features have been added during the development of 0.1.x:

    Better streaming support via the Event Streaming API.
    Standardized tool calling support
    A standardized interface for structuring output
    @chain decorator to more easily create RunnableLambdas
    https://python.langchain.com/docs/expression_language/how_to/inspect/
    In Python, better async support for many core abstractions (thank you @cbornet!!)
    Include response metadata in AIMessage to make it easy to access raw output from the underlying models
    Tooling to visualize your runnables or your langgraph app
    Interoperability of chat message histories across most providers
    Over 20+ partner packages in python for popular integrations

What’s coming to LangChain?

    We’ve been working hard on langgraph. We will be building more capabilities on top of it and focusing on making it the go-to framework for agent architectures.
    Vectorstores V2! We’ll be revisiting our vectorstores abstractions to help improve usability and reliability.
    Better documentation and versioned docs!
    We’re planning a breaking release (0.3.0) sometime between July-September to upgrade to full support of Pydantic 2, and will drop support for Pydantic 1 (including objects originating from the v1 namespace of Pydantic 2).

What changed?

Due to the rapidly evolving field, LangChain has also evolved rapidly.

This document serves to outline at a high level what has changed and why.
TLDR

As of 0.2.0:

    This release completes the work that we started with release 0.1.0 by removing the dependency of langchain on langchain-community.
    langchain package no longer requires langchain-community . Instead langchain-community will now depend on langchain-core and langchain .
    User code that still relies on deprecated imports from langchain will continue to work as long langchain_community is installed. These imports will start raising errors in release 0.4.x.

As of 0.1.0:

    langchain was split into the following component packages: langchain-core, langchain, langchain-community, langchain-[partner] to improve the usability of langchain code in production settings. You can read more about it on our blog.

Ecosystem organization

By the release of 0.1.0, LangChain had grown to a large ecosystem with many integrations and a large community.

To improve the usability of LangChain in production, we split the single langchain package into multiple packages. This allowed us to create a good foundation architecture for the LangChain ecosystem and improve the usability of langchain in production.

Here is the high level break down of the Eco-system:

    langchain-core: contains core abstractions involving LangChain Runnables, tooling for observability, and base implementations of important abstractions (e.g., Chat Models).
    langchain: contains generic code that is built using interfaces defined in langchain-core. This package is for code that generalizes well across different implementations of specific interfaces. For example, create_tool_calling_agent works across chat models that support tool calling capabilities.
    langchain-community: community maintained 3rd party integrations. Contains integrations based on interfaces defined in langchain-core. Maintained by the LangChain community.
    Partner Packages (e.g., langchain-[partner]): Partner packages are packages dedicated to especially popular integrations (e.g., langchain-openai, langchain-anthropic etc.). The dedicated packages generally benefit from better reliability and support.
    langgraph: Build robust and stateful multi-actor applications with LLMs by modeling steps as edges and nodes in a graph.
    langserve: Deploy LangChain chains as REST APIs.

In the 0.1.0 release, langchain-community was retained as required a dependency of langchain.

This allowed imports of vectorstores, chat models, and other integrations to continue working through langchain rather than forcing users to update all of their imports to langchain-community.

For the 0.2.0 release, we’re removing the dependency of langchain on langchain-community. This is something we’ve been planning to do since the 0.1 release because we believe this is the right package architecture.

Old imports will continue to work as long as langchain-community is installed. These imports will be removed in the 0.4.0 release.

To understand why we think breaking the dependency of langchain on langchain-community is best we should understand what each package is meant to do.

langchain is meant to contain high-level chains and agent architectures. The logic in these should be specified at the level of abstractions like ChatModel and Retriever, and should not be specific to any one integration. This has two main benefits:

    langchain is fairly lightweight. Here is the full list of required dependencies (after the split)

    python = ">=3.8.1,<4.0"
    langchain-core = "^0.2.0"
    langchain-text-splitters = ">=0.0.1,<0.1"
    langsmith = "^0.1.17"
    pydantic = ">=|=========================================================================================|
1,<3"
    SQLAlchemy = ">=1.4,<3"
    requests = "^2"
    PyYAML = ">=5.3"
    numpy = "^1"
    aiohttp = "^3.8.3"
    tenacity = "^8.1.0"
    jsonpatch = "^1.33"

    langchain chains/agents are largely integration-agnostic, which makes it easy to experiment with different integrations and future-proofs your code should there be issues with one specific integration.

There is also a third less tangible benefit which is that being integration-agnostic forces us to find only those very generic abstractions and architectures which generalize well across integrations. Given how general the abilities of the foundational tech are, and how quickly the space is moving, having generic architectures is a good way of future-proofing your applications.

langchain-community is intended to have all integration-specific components that are not yet being maintained in separate langchain-{partner} packages. Today this is still the majority of integrations and a lot of code. This code is primarily contributed by the community, while langchain is largely written by core maintainers. All of these integrations use optional dependencies and conditional imports, which prevents dependency bloat and conflicts but means compatible dependency versions are not made explicit. Given the volume of integrations in langchain-community and the speed at which integrations change, it’s very hard to follow semver versioning, and we currently don’t.

All of which is to say that there’s no large benefits to langchain depending on langchain-community and some obvious downsides: the functionality in langchain should be integration agnostic anyways, langchain-community can’t be properly versioned, and depending on langchain-community increases the vulnerability surface of langchain.

For more context about the reason for the organization please see our blog: https://blog.langchain.dev/langchain-v0-1-0/"""

In [3]:
def clean_cuda():
    torch.cuda.empty_cache()

In [2]:
import os, torch
os.environ['CUDA_VISIBLE_DEVICES'] = str(5)

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

model_id = "microsoft/Phi-3.5-mini-instruct"
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, 
        bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # Tokenizer initialization
    
model = AutoModelForCausalLM.from_pretrained(
    	model_id, 
    	torch_dtype="auto", 
    	trust_remote_code=True,
        quantization_config=bnb_config
	)
model.to(0)
tokenizer = AutoTokenizer.from_pretrained(model_id)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You shouldn't move a model that is dispatched using accelerate hooks.


'microsoft/Phi-3.5-mini-instruct'

In [4]:
system_command = "You are a summarizer."
user_command = text

messages = [{"role": "system", "content": system_command},
		        {"role": "user", "content": user_command}]

generation_args = {
        "use_cache": True,
        "max_new_tokens": 5000,
        #"temperature": 0.7,
        "repetition_penalty": 1.1,
        #"do_sample": True  # Enable sampling to use temperature
    }
pipe = pipeline(
		"text-generation",
		model=model,
		tokenizer=tokenizer,
	)
	    
output = pipe(messages, **generation_args)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


In [5]:
print(output[-1]['generated_text'][-1]['content'])

 **Overview of LangChain v0.2**

With the introduction of LangChain v0.2, several key improvements were implemented aimed at enhancing user experience, streamlining operations, and preparing the platform's infrastructure for upcoming developments. Below is an overview highlighting both existing enhancements and forthcoming updates within the software suite.

### New Features Added During Development of Previous Versions:

1. Enhanced event handling capability provided by improved `Event Streaming API`.
2. Established uniformity in tools invocation procedures (`standardized tool calling`).
3. Created consistent structures for outputs (`a standardized interface for structuring output`).
4. Introduced `@chain` syntax simplifying creation of executable lambda functions (`Runnable Lambdas`).
5. Improved introspection methods available directly in Python language constructors (`https://python.langchain.com/docs/expression_language/how_to/inspect/` link included).
6. Augmentation of asynchron

In [6]:
from IPython.display import Markdown, display

display(Markdown((output[-1]['generated_text'][-1]['content'])))

 **Overview of LangChain v0.2**

With the introduction of LangChain v0.2, several key improvements were implemented aimed at enhancing user experience, streamlining operations, and preparing the platform's infrastructure for upcoming developments. Below is an overview highlighting both existing enhancements and forthcoming updates within the software suite.

### New Features Added During Development of Previous Versions:

1. Enhanced event handling capability provided by improved `Event Streaming API`.
2. Established uniformity in tools invocation procedures (`standardized tool calling`).
3. Created consistent structures for outputs (`a standardized interface for structuring output`).
4. Introduced `@chain` syntax simplifying creation of executable lambda functions (`Runnable Lambdas`).
5. Improved introspection methods available directly in Python language constructors (`https://python.langchain.com/docs/expression_language/how_to/inspect/` link included).
6. Augmentation of asynchronous programming paradigms due to contributions such as cbornet (@cbornet), leading to smoother execution flow.
7. Capabilities allowing inclusion of detailed responses alongside messages sent out by artificial intelligence systems (`Include response metadata in AIMessage`).
8. Visualization options introduced for inspecting chain workflows (`Tooling to visualize your runnables`) along with application design patterns (`your langgraph apps`).
9. Cross-provider compatibility achieved among messaging history records ensuring seamless communication experiences (`Interoperability of chat message histories`).
10. Expansion beyond twenty partnerships established specifically targeting widespread adoption scenarios (`over 20+ partner packages...for popular integrations`).

### Upcoming Developments Expected With Release Version 0.3.0:

1. Further advancement planned for 'langgraph', setting its stage as central technology supporting complex agents interfacing with Large Language Models (LLMs).
2. Revisition and refinement process underway for vectors stores known as `'VectorStores'` anticipated to boost ease of usage and system resilience ('V2').
3. Committed effort towards comprehensive guidance material including updated documents reflective of current standards ("Better documentation").
4. Anticipate significant restructure occurring around Phantom dBase; transition expected toward complete embracement of newer specification set released after initial phase completion - namely `"Pydantic 2"` without retaining older ones linked backwardly to earlier releases.

### Key Transition Highlights From Prior State To Present Configuration:

A notable shift occurred whereby previous dependence upon `__init__.py`, previously located inside subdirectories named `_version`, got eliminated post-release 0.1.0. Consequently, direct import statements referencing modules formerly contained within said directory became obsolete unless explicitly stated otherwise. Specifically regarding legacy utilities tied to old module namespaces—such as importing entities originally sourced from `langchain-community`'s domain—this practice persists until subsequent major iteration milestones signal changes requiring adjustments elsewhere in dependent projects. It must be noted though, once past certain threshold points marked by later iterations, particularly indicated here as point `.4.0`, alterations would become mandatory if continued operation hinges on prior installations.

#### Summary TLDR:

From the perspective of evolutionary progressiveness inherent to technological platforms operating dynamically environments, LangChain exhibits continuous adaptation reflected throughout various stages culminating presently at verison mark '.2'. Noteworthy strides include dissociation from former communal repository prerequisites thereby fostering greater autonomy whilst preserving operative continuance contingent upon extant installation conditions. Such transitions underscore commitment to maturation trajectory geared towards optimizing end-user engagement coupled with fortified technical scaffoldings poised against impending expansions delineated henceforward.

---
*Note: While attempting to encapsulate extensive information succinctness may result in condensed descriptions necessitating further exploration outside presented textual summary.*

In [1]:
as

SyntaxError: invalid syntax (1239779345.py, line 1)